In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
import re
import time
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clapper-massive-rotten-tomatoes-movies-and-reviews/rotten_tomatoes_movies.csv
/kaggle/input/clapper-massive-rotten-tomatoes-movies-and-reviews/rotten_tomatoes_movie_reviews.csv
/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


# Parte 1: Búsqueda lineal en documentos

## Actividad

Se te proporcionará un dataset con reviews de películas.

Escribe una función que:

* Lea todos los documentos.
* Busque una palabra ingresada por el usuario.
* Muestre en qué documentos aparece la palabra.


In [28]:
#Lectura y visualizacion del dataset movie_data
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [29]:
#Esta funcion devuelve las filas que contengan la query ingresada por el usuario
def hasquery(query, docs):
    mask = docs.str.contains(query, na=False)
    resultados = docs[mask]
    return resultados

In [41]:
hasquery('good', df['review'])

4        Petter Mattei's "Love in the Time of Money" is...
12       So im not a big fan of Boll's work but then ag...
13       The cast played Shakespeare.<br /><br />Shakes...
14       This a fantastic movie of three prisoners who ...
19       An awful film! It must have been up against so...
                               ...                        
49986    This movie is a disgrace to the Major League F...
49988    When I first tuned in on this morning news, I ...
49995    I thought this movie did a down right good job...
49997    I am a Catholic taught in parochial elementary...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 18771, dtype: object

# Parte 2: Construcción de un índice invertido

## Actividad

Escribe un programa que:


* Recorra todos los documentos.
* Construya un índice invertido, es decir, un diccionario donde:
  * Cada palabra clave apunta a una lista de documentos donde aparece.


Escribe una nueva función de búsqueda que:


* Consulte directamente el índice para encontrar los documentos relevantes.
* Sea mucho más rápida que la búsqueda lineal.


In [31]:
#Función para procesar y normalizar el texto
def procesar_texto(texto):
    #Convertir todo a minúsculas y eliminar caracteres no alfabéticos
    
    #Convertir a minúsculas
    texto = texto.lower() 
    #Eliminar caracteres no alfabéticos
    texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)  
    return texto

In [32]:
#Función para construir el índice invertido
def construir_indice_invertido(df):
    indice_invertido = defaultdict(list)

    #Recorrer los documentos en df
    for index, row in df.iterrows():
        id_documento = index
        texto = row['review']
        texto_procesado = procesar_texto(texto)

        #Dividir el texto en palabras únicas
        palabras = set(re.findall(r'\b\w+\b', texto_procesado))

        #Agregar cada palabra al índice invertido
        for palabra in palabras:
            indice_invertido[palabra].append(id_documento)
    
    return indice_invertido



In [34]:
#Construir el índice invertido
indice = construir_indice_invertido(df)


In [46]:
#Funcion para buscar palabras usando el indice invertido
def buscar_palabra(palabra, indice_invertido, df):
    #Convertir la palabra a minuscula
    palabra = palabra.lower()
    #Guarda en documentos las filas que contengan la palabra
    documentos = indice_invertido.get(palabra, [])
    print(f"🔍 La palabra '{palabra}' aparece en {len(documentos)} documento(s):\n")
    

In [47]:
#Ejecutar la funcion para obtener el numero de documentos que tienen la palabra ingresada
buscar_palabra("good",indice, df)

🔍 La palabra 'good' aparece en 18506 documento(s):



### NOTA
Existe una discrepancia entre los documentos recuperados y los buscados con el anterior metodo, existen filas done la palabra buscada ya no aparece, es decir, EL INDICE ESTA MAL HECHO

# Parte 3: Evaluación de tiempos de búsqueda

## Actividad

Realiza la búsqueda de varias palabras usando:

* Corpus pequeño.
* Corpus grande.

Mide el tiempo de ejecución:

* Para búsqueda lineal.
* Para búsqueda usando índice invertido.
* Grafica o presenta los resultados en una tabla comparativa.


In [48]:
#Corpus Grande
#Lectura y visualizacion del dataset movie_data
df_large = pd.read_csv("/kaggle/input/clapper-massive-rotten-tomatoes-movies-and-reviews/rotten_tomatoes_movie_reviews.csv")
df_large.head()

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,beavers,1145982,2003-05-23,Ivan M. Lincoln,False,3.5/4,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,http://www.deseretnews.com/article/700003233/B...
1,blood_mask,1636744,2007-06-02,The Foywonder,False,1/5,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,http://www.dreadcentral.com/index.php?name=Rev...
2,city_hunter_shinjuku_private_eyes,2590987,2019-05-28,Reuben Baron,False,NaN,fresh,CBR,The choreography is so precise and lifelike at...,POSITIVE,https://www.cbr.com/city-hunter-shinjuku-priva...
3,city_hunter_shinjuku_private_eyes,2558908,2019-02-14,Matt Schley,False,2.5/5,rotten,Japan Times,The film's out-of-touch attempts at humor may ...,NEGATIVE,https://www.japantimes.co.jp/culture/2019/02/0...
4,dangerous_men_2015,2504681,2018-08-29,Pat Padua,False,NaN,fresh,DCist,Its clumsy determination is endearing and some...,POSITIVE,http://dcist.com/2015/11/out_of_frame_dangerou...


In [54]:
#Obtencion del tiempo de ejecucion con la funcion hasquery para el corpus pequeño df
inicio = time.time()
hasquery('good', df['review'])
fin = time.time()
print("Función hasquery tardo", fin - inicio)

Función hasquery tardo en (segundos) 0.06464266777038574


In [55]:
#Obtencion del tiempo de ejecucion con la funcion hasquery para el corpus grande df_large
inicio = time.time()
hasquery('good', df_large['reviewText'])
fin = time.time()
print("Función hasquery tardo", fin - inicio)

Función hasquery tardo 0.6643712520599365


In [56]:
#Función para procesar y normalizar el texto en el corpus grande
def procesar_texto(texto):
    #Convertir todo a minúsculas y eliminar caracteres no alfabéticos
    
    #Convertir a minúsculas
    texto = texto.lower() 
    #Eliminar caracteres no alfabéticos
    texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)  
    return texto

In [57]:
#Función para construir el índice invertido con el corpus grande
def construir_indice_invertido_grande(df):
    indice_invertido_grande = defaultdict(list)

    #Recorrer los documentos en df
    for index, row in df.iterrows():
        id_documento = index
        texto = row['reviewText']
        texto_procesado = procesar_texto(texto)

        #Dividir el texto en palabras únicas
        palabras = set(re.findall(r'\b\w+\b', texto_procesado))

        #Agregar cada palabra al índice invertido
        for palabra in palabras:
            indice_invertido_grande[palabra].append(id_documento)
    
    return indice_invertido

In [58]:
#Construir el índice invertido grande
indice_grande = construir_indice_invertido_grande(df_large)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
#Obtencion del tiempo de ejecucion con la funcion buscar_palabra para el corpus pequeño df


In [ ]:
#Obtencion del tiempo de ejecucion con la funcion buscar_palabra para el corpus grande df_large
